In [1]:
# !pip install -U datasets

In [2]:
# !sudo -H pip install -q transformers --upgrade

In [5]:
# !pip uninstall autoawq -y

In [ ]:
# !pip install autoawq

In [3]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [4]:
# !pip install transformers==4.35.2

In [5]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [6]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [8]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
import gc
gc.collect()


31

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [41:02<00:00, 76.96s/it]


In [ ]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/llmmodels/quantized_model/awq_mistral/tokenizer_config.json',
 '/llmmodels/quantized_model/awq_mistral/special_tokens_map.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.model',
 '/llmmodels/quantized_model/awq_mistral/added_tokens.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.json')

### Inference on quantized model

In [9]:
from transformers import pipeline
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [10]:
model_id = quant_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")

In [16]:
import numpy as np
np.seed(100)

AttributeError: module 'numpy' has no attribute 'seed'

In [13]:
import time
start = time.time()
output2 = pipe("what is art",
               max_new_tokens=500,
               num_return_sequences=1,
               do_sample=True,
               top_k=50,
               top_p=0.95)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 21.065829277038574


In [14]:
print(output2)

what is art, and what is not art. As one of the most important figures in art history, it is no wonder that he is known as the “Father of Art Criticism”.

The concept of modern art has undergone a significant number of metamorphoses. In his early stage, this form of art was characterized by movements and trends that sought to change traditional conceptions and ideas. During this period, artists experimented in new ways to express their ideas. After the end of World War I, a new type of art began to emerge. This type of art was marked by an intense period of growth in which artists experimented and changed traditional art forms.

Despite his many accomplishments, Clement Greenberg’s reputation as the “father of art criticism” has been a source of contention and controversy. There is a wide range of opinions about his contributions to the field, but most agree that he was a pioneer of the concept. In the 20th century, the definition of art has undergone a number of changes. The term “art

In [12]:
print(output2)

what is art, why we need it, and how it matters. We learn to value, enjoy, and experience art in all its forms. We begin by focusing on artistic creation and production, as we learn to create our own works of art. We then move on to look at art as communication and expression of cultural, social, and spiritual concerns.

### Introduction: Art in All Its Forms

Arts are the things that are usually made or created by a particular person or a group of people or a single human being, who is the originator. These human-made things can be used to express emotions and to create a meaningful message.

A person can create a great work of art, but at the same time, he or she can also destroy it. For instance, the Mona Lisa is a great work of art.

I think the most common form of art is the visual arts. I also believe the most common form of art that people are likely to encounter is the verbal arts.

### Artistic Creativity: Creative Expression and Communication

Creativity is not the same as th

### Inference on un-quantized model

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

In [ ]:
# model_path = '/data/quantization-trials/merged-model'/

In [4]:
model_id = model_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto",)
# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# def predict_from_normal(user_query):
#     _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
#     outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
#     output = tokenizer.decode(outputs[0])
#     return output

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
# Using unquant model
import time
start = time.time()
output2 = pipe("what is art", max_new_tokens=1000, num_return_sequences=1)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 102.65577578544617


In [14]:
print(output2)

what is art?

Art is a form of expression. It is a way to communicate ideas, emotions, and experiences through various mediums such as painting, sculpture, music, dance, and literature. Art can be created for a variety of reasons, including personal expression, social commentary, or simply for aesthetic pleasure.

## What is the purpose of art?

The purpose of art is to express the artist’s emotions, ideas, and experiences. It can also be used to communicate with others, to tell a story, or to simply make something beautiful. Art can be a form of self-expression, a way to connect with others, or a way to explore the world around us.

## What is the importance of art?

Art is important because it allows us to express ourselves in ways that words cannot. It can be a form of self-expression, a way to communicate with others, or a way to explore the world around us. Art can also be a form of therapy, helping us to process our emotions and experiences.

## What is the role of art in society

In [ ]:
model

In [ ]:
quant_model